In [3]:
import sys
sys.version

'3.10.2 (main, Sep 15 2022, 23:28:12) [Clang 15.0.0 (https://github.com/llvm/llvm-project 7effcbda49ba32991b8955821b8f'

# Задание 1

(**NB.** для запуска примеров кода нужен Python версии не ниже **3.10**, допускается использование других версий, в этом случае нужно самостоятельно избавиться от конструкции `match`).

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`: 

In [4]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)    

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__
 

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d 

Поддерживаются две операции - сложение и умножение. Применить можно так:

In [5]:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

f_diff = diff(f)

# значение производной в точке x = 2
f_diff(2)



22.0

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

In [92]:
from dataclasses import dataclass
from typing import Union, Callable, Tuple
from numbers import Number
from numpy import sign, isclose

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)    

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__
    
    def __sub__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value - o_value, self.d - o_d)
            case Number():
                return Dual(self.value - float(other), self.d)
            
    def __rsub__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(o_value - self.value, o_d - self.d)
            case Number():
                return Dual(float(other) - self.value, self.d)
    
    def __neg__(self) -> "Dual":
        return Dual(-self.value, -self.d)
    
    def __pos__(self) -> "Dual":
        return Dual(self.value, self.d)
    
    def __abs__(self) -> "Dual":
        if self.value == 0:
            raise ValueError('value should be non-zero')
        
        return Dual(abs(self.value), self.d * sign(self.value))
    
    def __invert__(self) -> "Dual":
        return Dual(~self.value, ~self.d)
    
    def __truediv__(self, other: Union["Dual", Number]) -> "Dual":
        return Dual._divide(self, other)
    
    def __rtruediv__(self, other: Union["Dual", Number]) -> "Dual":
        return Dual._divide(other, self)
    
    def __pow__(self, other: Number) -> "Dual":
        if self.value == 0:
            raise ValueError('value should be non-zero')
            
        return Dual(self.value**other, self.d * other * self.value**(other - 1)) 
    
    def is_close_to(self, other: 'Dual') -> bool:
        if not isinstance(other, Dual):
            return false
        
        return isclose(self.d, other.d) and isclose(self.value, other.value)
    
    @staticmethod
    def _divide(numerator: Union["Dual", Number], denominator: Union["Dual", Number]) -> "Dual":
        n_val, n_d = Dual.parse(numerator)
        d_val, d_d = Dual.parse(denominator)
        
        if d_val == 0:
            raise ZeroDivisionError('value of denominator should be non-zero')
        
        val_temp = n_val / d_val
        d_temp = (n_d * d_val - n_val * d_d) / (d_val * d_val)
        return Dual(val_temp, d_temp)
    
    @staticmethod    
    def parse(obj: Union["Dual", Number]) -> Tuple:
        return (obj.value, obj.d) if isinstance(obj, Dual) else (float(obj), .0)
        

In [63]:
from random import randint, random
import functools

class TestDual:
    PAIRS_GEN_OFFSET = -0.5
    PAIRS_GEN_SCALE  = 4.0
    TEST_REPETITION  = 10
    SPECIFIC_PAIRS   = [(1.0, 0.0), 
                        (0.0, 1.0), 
                        (0.0, 0.0)]
    
    @staticmethod
    def test_all():
        print("TestDual / test_all()")
        TestDual.test_value_errors()
        
        TestDual.sum_and_neg()
        TestDual.mul_and_div()
        TestDual.pow_and_mul()
    
    @staticmethod
    def test_value_errors():
        assert TestDual.abs_value_error() 
        assert TestDual.div_value_error()
        assert TestDual.pow_value_error() 
    
    @staticmethod
    def abs_value_error() -> bool:
        try:
            abs(Dual(0.0, 1.0))
            return False
        except ValueError:
            return True
        
    @staticmethod
    def div_value_error() -> bool:
        try:
            Dual(1.0, 1.0) / Dual(0.0, 1.0)
            return False
        except ZeroDivisionError:
            return True
        
    @staticmethod
    def pow_value_error() -> bool:
        try:
            Dual(0.0, 1.0)**1
            return False
        except ValueError:
            return True
    
    @staticmethod
    def sum_and_neg():
        for first, second in TestDual.get_pairs(TestDual.TEST_REPETITION):
                sum_val = Dual(first, second) + (-Dual(second, first))
                expect_val = Dual(first - second, second - first)
                assert sum_val.is_close_to(expect_val), f'not equal with {Dual(first, second)} and {-Dual(second, first)};\nsum: {sum_val}, exp: {expect_val}'
    
    @staticmethod
    def mul_and_div():
        for first, second in TestDual.get_pairs(TestDual.TEST_REPETITION):
            if first == 0 or second == 0:
                continue

            mul_val = Dual(first, second) * Dual(second, first)
            div_val = Dual(first, second) / (Dual(1, 0) / Dual(second, first))
            assert mul_val.is_close_to(div_val), f'not equal with {Dual(first, second)} and {Dual(second, first)};\nmul: {mul_val}, div: {div_val}'
    
    @staticmethod
    def pow_and_mul():
        for first, second in TestDual.get_pairs(TestDual.TEST_REPETITION):
            power = randint(1, 6)
                
            if first == 0:
                continue
            pow_val = Dual(first, second) ** power
            mul_val = functools.reduce(lambda a, b : a * b, [Dual(first, second)] * power)

            assert pow_val.is_close_to(mul_val), f'not equal with {Dual(first, second)} and power: {power};\npow: {pow_val}, mul: {mul_val}'
                
    @staticmethod  
    def get_pairs(common_pairs_number : int) -> Tuple:
        return tuple([((random() + TestDual.PAIRS_GEN_OFFSET) * TestDual.PAIRS_GEN_SCALE, 
                           (random() + TestDual.PAIRS_GEN_OFFSET) * TestDual.PAIRS_GEN_SCALE) 
                      for _ in range(common_pairs_number)] + TestDual.SPECIFIC_PAIRS)
        

In [64]:
TestDual.test_all()

TestDual / test_all()


## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [96]:
import math

def exp(input_value : Union["Dual", Number]):
    match input_value:
        case Dual(value, d):
            return Dual(math.exp(value), d * math.exp(value))
        case Number():
            return math.exp(input_value)

def cos(input_value : Union["Dual", Number]):
    match input_value:
        case Dual(value, d):
            return Dual(math.cos(value), -d * math.sin(value))
        case Number():
            return math.cos(input_value)

def sin(input_value : Union["Dual", Number]):
    match input_value:
        case Dual(value, d):
            return Dual(math.sin(value), d * math.cos(value))
        case Number():
            return math.sin(input_value)
        
def log(input_value : Union["Dual", Number]):
    match input_value:
        case Dual(value, d):
            if value <= 0:
                raise ValueError('value should be positive')
            return Dual(math.log(value), d / value)
        case Number():
            if input_value <= 0:
                raise ValueError('value should be positive')
            return math.log(input_value)
    
    
class TestDualExtended(TestDual):
    @staticmethod 
    def test_all():
        TestDual.test_all()
        
        TestDualExtended.sin_and_cos()
        TestDualExtended.log_and_exp()
        
    @staticmethod 
    def sin_and_cos():
        one = Dual(1.0, 0.0)
        for first, second in TestDual.get_pairs(TestDual.TEST_REPETITION):
            if first == 0:
                continue
            
            dual = Dual(first, second)
            assert (cos(dual)**2 + sin(dual)**2).is_close_to(one), f'not equal 1 with {dual}; cos^2: {cos(dual)**2}, sin^2: {sin(dual)**2}'
            
    @staticmethod       
    def log_and_exp():
        for first, second in TestDual.get_pairs(TestDual.TEST_REPETITION):
            dual = Dual(first, second)
            assert log(exp(dual)).is_close_to(dual), f'not equal with self: {dual}; log(exp(dual))={log(exp(dual))}'
            

In [66]:
TestDualExtended.test_all()

TestDual / test_all()


## Задание 1.3 (3 балла)
Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+5 баллов**)

In [5]:
from scipy.misc import derivative

def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

derivative(f, 2.)

22.0

In [123]:
from scipy.misc import derivative
import numpy as np
import numpy.typing as npt
from typing import Union, Callable

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d 

class CompareDualDerivativeWithNumerical:
    
    @staticmethod 
    def test_by_points(f : Callable[[float], float]):
        print('CompareDualDerivativeWithNumerical / test_by_points(f)')
        dual_derivative = diff(f)
        
        not_equality_interval = [None, None]
        d = []
        for value in CompareDualDerivativeWithNumerical.GetPoints(f, True):
            if np.isclose(dual_derivative(value), derivative(f, value, dx=1e-6)):
                continue
            
            if not not_equality_interval[0]:
                not_equality_interval[0] = value
            else:
                not_equality_interval[1] = value
            d.append(abs(dual_derivative(value) - derivative(f, value, dx=1e-6)))
        
        if not_equality_interval[0]:
            print(f'is not equal from {not_equality_interval[0]} to {not_equality_interval[1]}, avg dif={sum(d) / len(d)}')
    
    @staticmethod
    def GetPoints(f : Callable[[float], float] = None, isLog=False) -> np.ndarray:
        values = np.linspace(-2, 2, 100)
        temp_values = []
        if f:
            for i in range(len(values)):
                try:
                    f(values[i])
                    temp_values.append(values[i])
                except:
                    continue
            values = np.array(temp_values)
        
        if isLog:
            from_to = f'from {round(values[0], 2)} to {round(values[-1], 2)}' if len(values) > 1 else '' 
            print(f'values number: {len(values)} {from_to}')
        return values
        
            
    

In [101]:
def f_first(x: float) -> float:
    return 5 * x * x + 2 * x + 2

def f_second(x: float) -> float:
    return cos(x) + log(x) * 2

CompareDualDerivativeWithNumerical.test_by_points(f_first)
CompareDualDerivativeWithNumerical.test_by_points(f_second)

CompareDualDerivativeWithNumerical / test_by_points(f)
values number: 100
CompareDualDerivativeWithNumerical / test_by_points(f)
values number: 50


## Задание 1.4 (10 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

Генерацию случайных функций можно осуществить, например, двумя путями. 
1. Генерировать функцию в текстовом виде, зачем использовать встроенную функцию [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89 
```

2. Использовать стандартный модуль [ast](https://docs.python.org/3/library/ast.html), который позволяет во время выполнения программы манипулировать [Абстрактным Синтаксическим Деревом](https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B5_%D1%81%D0%B8%D0%BD%D1%82%D0%B0%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).
Например, выражение 

```python
func = lambda x: 2 * x + 5
```

Можно запрограммировать с помощью кода:

```python

expr = ast.Expression(
    body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),
        body=ast.BinOp(
            left=ast.BinOp(
                left=ast.Constant(value=2),
                op=ast.Mult(),
                right=ast.Name(id='x', ctx=ast.Load())
            ),
            op=ast.Add(),
            right=ast.Constant(value=5)
        )
    )
)

ast.fix_missing_locations(expr)

func = eval(compile(expr, filename="", mode="eval"))

assert func(42) == 89
```

При реализации нужно учитывать области допустимых значений функций.

In [114]:
import random 

operations = [' + ',' - ',' * ',' / ']
brackets_operations = ['(', 'cos(','sin(','log(','exp(']

gen = ''
init_depth = 0
MAX_DEPTH = 4
AFTER_CLOSING_BRACKETS_EXPRESION_CHANCE = 0.2

def generate_func(depth : int = 0, no_brackets : bool = False):
    generated = ''
    
    depth += 1
    need_closing = False
    
    if random.getrandbits(1) or no_brackets:
        generated += get_value()
        generated += random.choice(operations)
    else:
        generated += random.choice(brackets_operations)
        need_closing = True
        
    if depth <= MAX_DEPTH:
        generated += generate_func(depth, need_closing)
    else:
        generated += get_value()

    if need_closing:
        generated += ')'
    
    if random.random() < AFTER_CLOSING_BRACKETS_EXPRESION_CHANCE:
        generated += random.choice(operations)
        generated += generate_func(depth)

    return generated

def get_value() -> str:
    if random.getrandbits(1):
        return 'x'
    else:
        return str(random.randint(1, 99))

In [124]:
for i in range(20):
    gen_func = generate_func()
    print('\n' + gen_func)
    CompareDualDerivativeWithNumerical.test_by_points(eval("lambda x: " + gen_func))


42 + 87 - sin(47 * x - x) / exp(95 - 25)
CompareDualDerivativeWithNumerical / test_by_points(f)
values number: 100 from -2.0 to 2.0

x * 10 - 51 + x * (68)
CompareDualDerivativeWithNumerical / test_by_points(f)
values number: 100 from -2.0 to 2.0

x / 30 / 99 * 62 - exp(x)
CompareDualDerivativeWithNumerical / test_by_points(f)
values number: 100 from -2.0 to 2.0

x - cos(78 + x + 49 + 12) + exp(10 * sin(94 * 7))
CompareDualDerivativeWithNumerical / test_by_points(f)
values number: 100 from -2.0 to 2.0

80 - sin(10 - x - 50 / 85 * log(78 - x / x - 18)) * cos(96 - 2 + x)
CompareDualDerivativeWithNumerical / test_by_points(f)
values number: 100 from -2.0 to 2.0
is not equal from -2.0 to 2.0, avg dif=0.6761947765831828

log(x / cos(54 + (99)) * exp(x / 26) + log(43))
CompareDualDerivativeWithNumerical / test_by_points(f)
values number: 100 from -2.0 to 2.0

48 - x + cos(x / cos(26))
CompareDualDerivativeWithNumerical / test_by_points(f)
values number: 100 from -2.0 to 2.0
is not equal fro

## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [300]:
import inspect 

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d 

def diff_multy(func):
    arg_names = list(inspect.signature(func).parameters.keys())
    
    def diff_func(**args):
        values = []
        
        for arg in arg_names:
            temp_args = args.copy()
            temp_args[arg] = Dual(args[arg], 1.0)
            value = func(**temp_args)
            values.append(value.d)

        return values
    
    return diff_func

In [303]:
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  

f_diff = diff_multy(f)

print(f_diff(x=10, y=10, z=10))

[10.0, 5.0, 1.0]
